<a href="https://colab.research.google.com/github/usaidahmed01/Deep-Learning/blob/master/20Jan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train.shape
x_test.shape
class_names = ['airplane','automobile','bird','cat','deer',
    'dog','frog','horse','ship','truck']
plt.figure(figsize=(6,6))
for i in range(9):
  plt.subplot(3,3,i+1)
  plt.imshow(x_train[i])
  plt.title(class_names[y_train[i][0]])
  plt.axis('off')
plt.show()
#Normalization
x_train = x_train / 255.0
x_test = x_test / 255.0

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    x_train, y_train, epochs=10, validation_data=(x_test, y_test)
)
# layers.Conv2D(128, (3,3), activation='relu'),
# layers.MaxPooling2D(2,2),

# layers.Conv2D(32, (5,5), activation='relu')

predictions = model.predict(x_test)
predictions

y_pred = np.argmax(predictions, axis=1)
y_true = y_test.flatten()
wrong_indices = np.where(y_pred != y_true)[0]
print('Total wrong predictions', len(wrong_indices))
plt.figure(figsize=(8,8))
for i, idx in enumerate(wrong_indices[:9]):
  plt.subplot(3,3,i+1)
  plt.imshow(x_test[idx])
  plt.title(
      f'Pred: {class_names[y_pred[idx]]}\n'
      f'True: {class_names[y_true[idx]]}'
  )
  plt.axis('off')
plt.show()
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred)
import seaborn as sns
plt.figure(figsize=(10,8))
sns.heatmap(cm, xticklabels=class_names, yticklabels=class_names, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
#Data Augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)
history_aug = model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=10,
    validation_data=(x_test, y_test)
)
predictions = model.predict(x_test)
y_pred = np.argmax(predictions, axis=1)
y_true = y_test.flatten()
wrong_indices = np.where(y_pred != y_true)[0]
print('Total wrong predictions', len(wrong_indices))
#TRANSFER LEARNING
base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(32,32,3)
)
base_model.trainable=False
transfer_model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])


In [ ]:
y_pred = np.argmax(model.predict(x_test) , axis = 1)


In [ ]:
y_true = y_test.flatten()


In [ ]:
wrong_indices = np.where(y_true != y_true)[0]
print('Total wrong predictions ', len(wrong_indices))

In [ ]:
plt.figure(figsize = (8 , 8))
for i, idx in enumerate(wrong_indices[:9]):
  plt.subplot(3 , 3 , i + 1)
  plt.imshow(x_test[idx])
  plt.title(f'Pred:{class_names[y_pred[idx]]} \n'
  f'True: {class_names[y_true[idx]]}')
  plt.axis('off')
  plt.show()

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred , y_true)

plt.figure(figsize = (10 , 8))
sns.heatmap(cm, xticklabels = class_names, yticklabels = class_names ,cmap = 'Blues')
plt.xlabel('Predicted')
plt.ylabel('True')

In [ ]:
# Data Augmentation
  # means model is good but the data is unstructured

  datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rotation_range = 15, # camera angle
      width_shift_range = 0.1, # camera movement
      height_shift_range = 0.1,
      horizontal_flip = True, # Mirror image
      # vertical_flip = True
      # zoom_range = 0.1,
  )

In [ ]:
datagen.fit(x_train)
history_aug = model.fit(
    datagen.flow(x_train , y_train , batch_size = 64),
    epochs = 10,
    validation_data = (x_test , y_test)
)

In [ ]:
predictions = model.predict(x_test)
y_pred = np.argmax(predictions , axis = 1)
y_true = y_test.flatten()
wrong_indices = np.where(y_true != y_pred)[0]
print('Total wrong predictions ', len(wrong_indices))

In [ ]:
# Transfer Learning: This technique reuses a pre-trained model on a new, related task.
# It leverages knowledge gained from a large dataset (like ImageNet) to solve a new problem,
# especially when the new dataset is small.

# Load a pre-trained MobileNetV2 model.
# MobileNetV2 is a lightweight deep convolutional neural network architecture.
base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet', # Use weights pre-trained on the ImageNet dataset.
    include_top=False,  # Exclude the classifier (the top layers) from the original model.
                        # This allows us to add our own classification layers for the new task.
    input_shape=(32,32,3) # Define the input shape for our specific dataset (CIFAR-10 images are 32x32x3).
)

# Freeze the weights of the base model.
# This prevents the pre-trained weights from being updated during the training of our new layers.
# This is common in the initial stages of transfer learning to use the base model as a fixed feature extractor.
base_model.trainable=False

# Create a new sequential model on top of the pre-trained base model.
transfer_model = tf.keras.Sequential([
    base_model, # The frozen MobileNetV2 model acts as a feature extractor.
    layers.GlobalAveragePooling2D(), # A layer to reduce the spatial dimensions of the feature maps
                                     # to a single vector per feature map, effectively flattening them.
                                     # This is suitable for replacing the dense layers at the top.
    layers.Dense(128, activation='relu'), # A new fully connected hidden layer with ReLU activation.
    layers.Dense(10, activation='softmax') # The output layer with 10 units (for 10 CIFAR-10 classes)
                                         # and softmax activation for multi-class classification.
])

# When to use these built-in models (Transfer Learning):
# 1. When you have a small dataset but a similar task has a large dataset available (e.g., ImageNet).
# 2. To leverage powerful, pre-trained feature extractors, saving time and computational resources.
# 3. To achieve good performance with limited training data.
# 4. As a strong starting point for fine-tuning (where you might later unfreeze some layers of the base model).
# 5. When the problem domain is similar to the domain the model was originally trained on.

In [ ]:
# agar pixels bht hi chotay ahin tou best hai ke maxpooling apply and if pixels barhay hain like 264 x 264 so we can use average pooling and all